<a href="https://colab.research.google.com/github/gpnlab/DL_BookClub/blob/master/notebooks/DeepLearningIllustrated/shallow_net_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No data download, tensorflow/pytorch installation, or modifications to the jupyter notebook required! Just follow the link and make sure to select a GPU as the hardware accelerator: 
```
Runtime > Change runtime type > Hardware accelerator > Select GPU
```

# Concrete Example of a Neural Network in TensorFlow

In this notebook, we test the installation and build a shallow neural network to classify handwritten digits.

## GPU-Jupyter

This Jupyterlab instance is running inside a remote container that is connected to the host machine graphical processing units (GPUs) via CUDA drivers. Note that the corresponding Nvidia graphics drivers must be installed in the host machine.

CUDA is a parallel computing platform and programming model developed by NVIDIA for general computing on GPUs.

This container has the CUDA 11.2 toolkit installed. The CUDA Toolkit from NVIDIA includes GPU-accelerated libraries, a compiler, development tools and the CUDA runtime.

The container also has installed the NVIDIA CUDA Deep Neural Network library (cuDNN) 8.0. cuDNN is a GPU-accelerated library of primitives for deep neural networks. cuDNN provides highly tuned implementations for standard routines such as forward and backward convolution, pooling, normalization, and activation layers.

### Test GPU connection

Using the following command, your GPU type and its NVIDIA System Management Interface (NVIDIA-SMI) driver version should be listed:

In [ ]:
!nvidia-smi

### Test if Tensorflow can access the GPU via CUDA

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(tf.config.list_physical_devices("CPU"))
print(tf.config.list_physical_devices("GPU"))
device_lib.list_local_devices()

### Performance test

Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numPy, PyTorch and Tensorflow with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.
$$H=X(X^TX)^{-1}X^T$$

In [1]:
import time
import numpy as np
import torch

#### 1) NumPy

In [2]:
x = np.random.rand(10000, 256)

In [3]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

17.2 s ± 366 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 2a) PyTorch on CPU

In [ ]:
device = torch.device("cpu")
x = torch.rand(10000, 256, device=device)
%timeit H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

#### 2b) Tensorflow on CPU

In [ ]:
with tf.device("CPU:0"):
    x = tf.random.normal([10000, 256])
    assert x.device.endswith("CPU:0")
    %timeit H = tf.matmul(tf.linalg.inv(tf.matmul(tf.transpose(x), x)), tf.transpose(x))

#### 3a) PyTorch on GPU via CUDA

In [ ]:
# let us run this cell only if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    x = torch.rand(10000, 256, device=device)
    %timeit H = x.mm((x.t().mm(x)).inverse()).mm(x.t())

#### 3b) Tensorflow on GPU via CUDA

In [ ]:
# let us run this cell only if GPU is available
if tf.config.list_physical_devices("GPU"):
    with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random.normal([10000, 256])
        assert x.device.endswith("GPU:0")
        %timeit H = tf.matmul(tf.linalg.inv(tf.matmul(tf.transpose(x), x)), tf.transpose(x))

## Classifying handwritten digits with a shallow neural network

### Load dependencies

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from matplotlib import pyplot as plt

### Load data

The MNIST dataset comes preloaded in tf.Keras in the form of a set of four NumPy arrays

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train.shape

In [ ]:
len(y_train)

In [ ]:
y_train[0:12]

In [ ]:
plt.figure(figsize=(5, 5))
for k in range(12):
    plt.subplot(3, 4, k + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[k], cmap="Greys")
    plt.xlabel(y_train[k])
plt.tight_layout()
plt.show()

In [ ]:
X_test.shape

In [ ]:
len(y_test)

In [ ]:
plt.imshow(X_test[0], cmap="Greys")

In [ ]:
y_test

### Preprocess data

In [ ]:
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

### Design neural network architecture

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(512, activation="relu"))
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
(512 * 784) + 512 + (10 * 512) + 10

### Configure model

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"]
)

### Train!

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=5,
    verbose=1,
    validation_data=(X_test, y_test),
)

### Performing inference

In [ ]:
test_digits = X_test
predictions = model.predict(test_digits)
predictions[0]

In [ ]:
np.argmax(predictions[0], axis=-1)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap="Greys")

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = "blue"
    else:
        color = "red"

    plt.xlabel(
        "{} {:2.0f}% ({})".format(
            predicted_label, 100 * np.max(predictions_array), true_label
        ),
        color=color,
    )


def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color("red")
    thisplot[true_label].set_color("blue")

In [ ]:
i = 0
plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plot_image(i, predictions[i], y_test, X_test)
plt.subplot(1, 2, 2)
plot_value_array(i, predictions[i], y_test)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plot_image(i, predictions[i], y_test, X_test)
plt.subplot(1, 2, 2)
plot_value_array(i, predictions[i], y_test)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions[i], y_test, X_test)
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
    plot_value_array(i, predictions[i], y_test)
plt.tight_layout()
plt.show()

### Evaluating model performance

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("\n Test accuracy: {:.2f}%".format(100 * test_acc))